In [1]:
import sqlite3
import sys
import requests_cache
import pandas as pd
import requests
import random
import string
from bs4 import NavigableString
from bs4 import BeautifulSoup as bs

In [2]:
sys.path.insert(0, "/Users/samdillard/python/github.com/samhld/nba_stats/app/")
requests_cache.install_cache()

In [3]:
from scraper import getters
from preprocessor import builders
from PandasBasketball.stats import player_stats
from PandasBasketball.stats import get_data_master
from PandasBasketball.errors import TableNonExistent
from preprocessor.builders import create_player_stats_df

In [4]:
# conn = sqlite3.connect("../app/dbs/all_players.db")
# c = conn.cursor()

In [5]:
# master = """SELECT name FROM sqlite_master WHERE type='table'"""
# c.execute(master).fetchall()

In [6]:
all_players = '''SELECT PTS FROM players_per_game WHERE Player = "LeBron James"'''
# c.execute(all_players).fetchall()

In [7]:
rename_table = '''ALTER TABLE `players_per_game` RENAME TO `players_per36`'''
# c.execute(rename_table)

In [8]:
# master = """SELECT name FROM sqlite_master WHERE type='table'"""
# c.execute(master).fetchall()

In [9]:
def _get_player_tables(url, stat):
    
    one_header_tables = ["totals", "per_minute", "per_poss", "advanced",
                        "playoffs_per_game", "playoffs_totals", "playoffs_per_minute",
                        "playoffs_per_poss", "playoffs_advanced"]
    
    two_header_tables = ["adj_shooting", "playoffs_shooting", "shooting", "pbp", "playoffs_pbp"]
    
    res = requests.get(url)
    
    if stat == "per_game":
        
        soup = bs(res.text, "lxml")
        table = soup.find("table", id="per_game")
        
    elif stat in one_header_tables or stat in two_header_tables:

        soup = bs(res.text, "lxml")
        comment_table = soup.find(text=lambda x: isinstance(x, NavigableString) and f'id="{stat}"' in x)
        soup = bs(comment_table, "lxml")
        table = soup.find("table", id=stat)
        
        if stat in two_header_tables:
            
            table.find("tr").decompose()

    else:
        raise TableNonExistent
    return table
        

In [57]:
from scraper.getters import _get_player_table
bron_url = "https://www.basketball-reference.com/players/j/jamesle01.html"
_get_player_table(bron_url, "playoffs_per_game")

<table class="row_summable sortable stats_table" data-cols-to-freeze="1,3" id="playoffs_per_game">
<caption>Playoffs Per Game Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="If listed as single number, the year the season ended.★ - Indicates All-Star for league.Only on regular season tables." class="poptip sort_default_asc center" data-stat="season" data-tip="If listed as single number, the year the season ended.&lt;br&gt;★ - Indicates All-Star for league.&lt;br&gt;Only on regular season tables." scope="col">Season</th>
<th aria-label="Player's age on February 1 of the season" class="poptip sort_default_asc center" data-stat="age" data-tip="Player's age on February 1 of the season" scope="col">Age</th>
<th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_id" data-tip

In [11]:
# no selenium way

res = requests.get(bron_url)
soup = bs(res.text, "html.parser")
comment_table = soup.find(text=lambda x: isinstance(x, NavigableString) and "id=shooting" in x)
comment_table
# soup = BeautifulSoup(comment_table, "lxml")
# table = soup.find("table")
# table

In [12]:
# chromedriver = "/Users/samdillard/Downloads/chromedriver"
# options = ChromeOptions()
# options.add_argument('--headless')
# driver = webdriver.Chrome(chromedriver, options=options)
# soup = bs(driver.page_source,'lxml')
# driver.quit()
# tables = soup.find_all('table')
# tables

In [13]:
bron_url = "https://www.basketball-reference.com/players/j/jamesle01.html"

In [34]:
def get_all_player_tables(url):
    
    supported_tables = ["per_game", "totals", "per_minute", "per_poss", "advanced",
                        "playoffs_per_game", "playoffs_totals", "playoffs_per_minute",
                        "playoffs_per_poss", "playoffs_advanced", "adj_shooting", 
                        "playoffs_shooting", "shooting", "pbp", "playoffs_pbp"]
    
    tables = []
    for table in supported_tables:
        tables.append(_get_player_table(url, table))
        
    return tables
        

In [35]:
bron_tables = get_all_player_tables(bron_url)

In [39]:
for i,table in enumerate(bron_tables):
    print(i, table.caption.text)

0 Per Game Table
1 Totals Table
2 Per 36 Minutes Table
3 Per 100 Poss Table
4 Advanced Table
5 Playoffs Per Game Table
6 Playoffs Totals Table
7 Playoffs Per 36 Minutes Table
8 Playoffs Per 100 Poss Table
9 Playoffs Advanced Table
10 Adjusted Shooting Table
11 Playoffs Shooting Table
12 Shooting Table
13 Play-by-Play Table
14 Playoffs Play-by-Play Table


In [56]:
get_data_master(bron_tables[12], "player").iloc[:,7:22]

,FG%,Dist.,,2P,0-3,3-10,10-16,16-3P,3P,,2P,0-3,3-10,10-16,16-3P
0,.417,11.2,,.855,.315,.168,.161,.211,.145,,.438,.604,.356,.313,.352
1,.472,12.1,,.817,.311,.137,.121,.248,.183,,.499,.725,.370,.324,.373
2,.480,12.4,,.792,.353,.092,.094,.253,.208,,.518,.689,.411,.304,.398
3,.476,12.2,,.809,.333,.125,.111,.241,.191,,.513,.725,.406,.361,.346
4,.484,12.2,,.781,.373,.090,.080,.238,.219,,.531,.706,.426,.303,.372
5,.489,13.4,,.762,.333,.092,.074,.263,.238,,.535,.728,.453,.286,.388
6,.503,13.4,,.747,.342,.082,.061,.262,.253,,.560,.745,.468,.344,.398
7,.510,12.7,,.812,.284,.172,.094,.262,.188,,.552,.744,.459,.443,.445
8,.531,11.5,,.873,.329,.136,.135,.272,.127,,.556,.758,.484,.481,.384
9,.565,11.2,,.812,.373,.133,.105,.202,.188,,.602,.776,.494,.415,.447
